In [1]:
import os
import numpy as np
from PIL import Image
import platform
from ctypes import *
import matplotlib.pyplot as plt

In [2]:
DATASET_FOLDER = "dataset"
TRAIN_SUBFOLDER = os.path.join(DATASET_FOLDER, "train")
TEST_SUBFOLDER = os.path.join(DATASET_FOLDER, "test")

TRAIN_ACTION_FOLDER= os.path.join(TRAIN_SUBFOLDER, "action")
TRAIN_COMEDY_FOLDER= os.path.join(TRAIN_SUBFOLDER, "comedie")

TEST_ACTION_FOLDER = os.path.join(TEST_SUBFOLDER, "action")
TEST_COMEDY_FOLDER= os.path.join(TEST_SUBFOLDER, "comedie")

In [3]:
def fill_x_and_y(folder, x_list, y_list, label):
    for file in os.listdir(folder):
        file_path = os.path.join(folder, file)
        image = Image.open(file_path)
        image = image.resize((8,8))
        im_arr = np.array(image).flatten()
        x_list.append(im_arr)
        y_list.append(label)
        
def import_dataset():
    X_train = []
    Y_train = []
    X_test = []
    Y_test = []
    fill_x_and_y(TRAIN_ACTION_FOLDER, X_train, Y_train, 1.0)
    fill_x_and_y(TRAIN_COMEDY_FOLDER, X_train, Y_train, -1.0)
    fill_x_and_y(TEST_ACTION_FOLDER, X_test, Y_test, 1.0)
    fill_x_and_y(TEST_COMEDY_FOLDER, X_test, Y_test, -1.0)
    
    return (np.array(X_train) / 255.0, np.array(Y_train)), (np.array(X_test)/255.0, np.array(Y_test))

In [4]:
(X_train, Y_train), (X_test, Y_test) = import_dataset()

In [5]:
print(len(X_train[0]))

192


In [6]:
path_to_shared_library = "target/debug/librust_lib.dylib"
path_to_shared_library_windows = "target\\debug\\rust_lib.dll"

if(platform.system()=='Windows'):
    my_lib = cdll.LoadLibrary(path_to_shared_library_windows)
else : 
    my_lib = cdll.LoadLibrary(path_to_shared_library)

In [25]:
d = [2, 2, 1]
arr_size = len(d)
arr_type = c_int * arr_size
native_arr = arr_type(*d)
my_lib.create_mlp_model.argtypes = [arr_type, c_int]
my_lib.create_mlp_model.restype = c_void_p

model = my_lib.create_mlp_model(native_arr,arr_size)

flattened_dataset_inputs = []
for p in X_train:
    flattened_dataset_inputs.append(p[0])
    flattened_dataset_inputs.append(p[1])

flattened_inputs_len = len(flattened_dataset_inputs)
flattened_outputs_len = len(Y_train)

inputs_type = c_float * flattened_inputs_len
outputs_type = c_float * flattened_outputs_len

my_lib.train_classification_stochastic_backdrop_mlp_model.argtypes = [c_void_p,inputs_type,outputs_type,c_float,c_int,c_int,c_int]
my_lib.train_classification_stochastic_backdrop_mlp_model.restype = None
inputs_native = inputs_type(*flattened_dataset_inputs)
outputs_native = outputs_type(*Y_train)

my_lib.train_classification_stochastic_backdrop_mlp_model(model,inputs_native,outputs_native,0.001,100000,flattened_inputs_len,flattened_outputs_len)

In [26]:
for p in X_test:
    p_type = c_float * len(p)
    break
my_lib.predict_mlp_model_classification.argtypes = [c_void_p, p_type,c_int]
my_lib.predict_mlp_model_classification.restype = POINTER(c_float)

predicted_outputs = [my_lib.predict_mlp_model_classification(model, p_type(*p),len(p))[0] for p in X_test]

In [27]:
print(predicted_outputs)

[0.0211044792085886, -0.12861762940883636, 0.036703646183013916, -0.5986766815185547, 0.4199751317501068, -0.5620144605636597, -0.0007690189522691071, 0.740168571472168, -0.22503869235515594, -0.07689607888460159, -0.3779132664203644, -0.7481273412704468, -0.5963156223297119, -0.34989261627197266, -0.3602322041988373]


In [11]:
my_lib.destroy_model.argtypes = [c_void_p]
my_lib.destroy_model.restype = None
my_lib.destroy_model(model)